# MayImmo POI
The goal of this notebook project is to gather all POI (Points of Interest) near a location in Mayotte
[Source](https://github.com/vaclavdekanovsky/data-analysis-in-examples/blob/master/Maps/Driving%20Distance/Driving%20Distance%20between%20two%20places.ipynb)

In [20]:
from geopandas.geodataframe import GeoDataFrame
import osmnx as ox
from osmnx.distance import euclidean_dist_vec
from pprint import pprint
import overpass
import geojson
import geopandas as gpd
from query import query
from pyproj import Proj
from geopy import distance

ox.config(use_cache=True, log_console=False)
api = overpass.API()

# step 1 : extract raw data from the dowloaded json
with open("./mayotte.json", mode="r") as f:
    data_load = geojson.load(f)["elements"]

gdf = gpd.GeoDataFrame(data_load)
gdf.crs = {"init": "epsg:4326"}

/home/eddine/Documents/project/Real Estate MLS/openstreetmap/venv/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [21]:
gdf["geometry"] = gpd.points_from_xy(gdf["lon"], gdf["lat"])
gdf = gdf.to_crs("EPSG:3857")
gdf.head(2)

,type,id,lat,lon,tags,bounds,nodes,geometry,members
0,node,100948360,-12.776235,45.233917,{'leisure': 'slipway'},NaN,NaN,POINT (5035416.618 -1434179.077),NaN
1,node,364411600,-12.778358,45.232845,"{'amenity': 'ferry_terminal', 'ferry': 'yes', ...",NaN,NaN,POINT (5035297.261 -1434421.489),NaN


In [22]:
# Process only type == node
gdf_node = gdf[gdf["type"]=="node"]
gdf_node.head(3)

,type,id,lat,lon,tags,bounds,nodes,geometry,members
0,node,100948360,-12.776235,45.233917,{'leisure': 'slipway'},NaN,NaN,POINT (5035416.618 -1434179.077),NaN
1,node,364411600,-12.778358,45.232845,"{'amenity': 'ferry_terminal', 'ferry': 'yes', ...",NaN,NaN,POINT (5035297.261 -1434421.489),NaN
2,node,364411621,-12.781689,45.257971,"{'leisure': 'slipway', 'name': 'Mamoudzou Ferry'}",NaN,NaN,POINT (5038094.331 -1434801.702),NaN


In [ ]:
# Sample 1 location and output POI around + distance
import requests
import json
lat, lon = -12.844411, 45.188579
r = requests.get(f"http://router.project-osrm.org/route/v1/car/{lon_1},{lat};{lon_2},{lat_2}?overview=false""")